In [1]:
import requests #package for http requests
import bs4 # package for html parsing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import timedelta, date

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# It creates the list of page links we will get the data from.
def create_links_2020(country):
    start_date = date(2020, 1, 1)
    end_date = date(2020,12,31)
    links = []
    dates = daterange(start_date, end_date)
    for single_date in daterange(start_date, end_date):
        links.append('https://spotifycharts.com/regional/' + country + '/daily/' + single_date.strftime("%Y-%m-%d"))
    return(links)

In [ ]:
def create_links_2017(country):
    start_date = date(2017, 1, 1)
    end_date = date(2017,12,31)
    links = []
    dates = daterange(start_date, end_date)
    for single_date in daterange(start_date, end_date):
        links.append('https://spotifycharts.com/regional/' + country + '/daily/' + single_date.strftime("%Y-%m-%d"))
    return(links)

In [9]:
def get_table(link):
    '''
    #rename later
    a function that gets a table of songs from the spotify charts
    with position, track, streams, and date
    '''
    r = requests.get(link)
    df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
    df = df_list[0]
    df=df.head(50) #get only top 50 songs, rather than 200
    #cleaning column names and dropping irrelevant columns
    cols= [x.lower() for x in df.columns]
    df.columns=cols
    df = df.rename(columns={'unnamed: 1': 'position'})
    df=df.drop(columns=["unnamed: 0","unnamed: 2"])
    #adding a date column by parsing the link
    df["date"]=link[-10:]
    df['date']=pd.to_datetime(df['date'])
    #ksjflksdf
    datasoup = bs4.BeautifulSoup(r.text, 'html.parser')
    aList=[]
    for data in datasoup.findAll('a'):
        aList.append(data)
    links=[]
    for x in aList:
        links.append(x.get('href'))
    #drop None types
    clean = filter(None, links)
    urls=[]
    for x in clean:
        if 'open.spotify.com/track/' in x:
            urls.append(x)
    df['url']=urls[:50]
    return df

In [16]:
def track_artist(df):
    '''
    takes "track" column, which contains both the artist and song and parses it into 2 columns: track_name and artist
    replaces "track" column with "track_name" and "artist" columns
    '''
    tracks=df.track.values
    songtitles=[]
    songartists=[]
    for song in tracks:
        index1=song.find("  by")
        index2=song.find("by ")
        track=song[:index1]
        songtitles.append(track)
        artist=song[index2+3:]
        songartists.append(artist)
    df["track_name"]=songtitles
    df["artist"]=songartists
    df=df.drop(columns=["track"])
    return df

In [43]:
uslinks=create_links("us")
testlink=uslinks[0]
test=get_table(testlink)
test=track_artist(test)
test['region']='us'
print(test.shape)
test.head()

(50, 7)


,position,streams,date,url,track_name,artist,region
0,1,1812103,2020-01-01,https://open.spotify.com/track/0nbXyq5TXYPCO7p...,The Box,Roddy Ricch,us
1,2,1400495,2020-01-01,https://open.spotify.com/track/696DnlkuDOXcMAn...,ROXANNE,Arizona Zervas,us
2,3,1186861,2020-01-01,https://open.spotify.com/track/21jGcNKet2qwijl...,Circles,Post Malone,us
3,4,1130123,2020-01-01,https://open.spotify.com/track/6Ozh9Ok6h4Oi1wU...,BOP,DaBaby,us
4,5,968612,2020-01-01,https://open.spotify.com/track/6Gg1gjgKi2AK4e0...,Bandit (with YoungBoy Never Broke Again),Juice WRLD,us


In [44]:
list=test["url"]
list[1]

'https://open.spotify.com/track/696DnlkuDOXcMAnKlTgXXK'

In [59]:
happy2020=pd.read_csv("happiness2020.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'happiness2020.csv'

In [45]:
region_id2020=['no', 'dk', 'is', 'ch', 'fi', 'nl', 'ca', 'nz', 'se', 'au', 'cr',
       'at', 'us', 'ie', 'de', 'be', 'lu', 'gb', 'cl', 'br', 'cz', 'ar',
       'mx', 'sg', 'uy', 'gt', 'pa', 'fr', 'tw', 'es', 'co', 'sk', 'my',
       'ec', 'sv', 'pl', 'it', 'jp', 'lt', 'lv', 'bo', 'pe', 'ee', 'tr',
       'py', 'ph', 'hu', 'id', 'do', 'gr', 'pt', 'hn']

In [46]:
spotify2020=pd.DataFrame()
for region in region_id2020:
    links=create_links_2020(region)
    for link in links:
        df=get_table(link)
        df=track_artist(df)
        df['region']=region
        spotify2020=spotify2020.append(df,ignore_index=True)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [47]:
spotify2020

,position,streams,date,url,track_name,artist,region
0,1,87442,2020-01-01,https://open.spotify.com/track/0sf12qNH5qcw8qp...,Blinding Lights,The Weeknd,no
1,2,73738,2020-01-01,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,Dance Monkey,Tones And I,no
2,3,66652,2020-01-01,https://open.spotify.com/track/2Pckb69I4iMmL24...,Jævlig,TIX,no
3,4,54797,2020-01-01,https://open.spotify.com/track/3ZCTVFBt2Brf31R...,everything i wanted,Billie Eilish,no
4,5,53728,2020-01-01,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Falling,Trevor Daniel,no
...,...,...,...,...,...,...,...
103695,46,81787,2020-09-05,https://open.spotify.com/track/5jyHAEYylcg1mQY...,Coca,"Ashafar, Mula B",nl
103696,47,80338,2020-09-05,https://open.spotify.com/track/7C1trmcQQ5n5RNy...,Some Say - Felix Jaehn Remix,Nea,nl
103697,48,80042,2020-09-05,https://open.spotify.com/track/3R02EuKIQgHKEGq...,Bad People,Kevin,nl
103698,49,80013,2020-09-05,https://open.spotify.com/track/2f0pn9DkEJwAzXA...,Someone To You,BANNERS,nl
